<a href="https://colab.research.google.com/github/jiyoung830/data_structure-lab/blob/master/Lec07_Serch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Multiplication Method

In [4]:
import math
def multiplication_hash(k,m,A=(math.sqrt(5)-1/2)):
  frac = k * A % 1
  return math.floor(m * frac)

2. Folding Method

In [10]:
import math
def folding_hash(k,m, group = 2):
  base = 10 ** group
  s = 0
  while k>0 :
    s+= k % base
    k //= base
  return s % m



3. Mid-squaare Method

In [8]:
import math
def  mid_square_hash(k,m):
  s = k ** 2
  mid = (len(str(s))//2)    # 가운데 위치
  return int(str(s)[mid:mid+2]) % m


In [16]:
m = 16
keys = [123,125,616]
print("key | mult | fold | mid")
for k in keys:
  print(f"{k} | {multiplication_hash(k,m)} | {folding_hash(k,m)} | {mid_square_hash(k,m)}")

key | mult | fold | mid
123 | 8 | 8 | 12
125 | 0 | 10 | 14
616 | 6 | 6 | 13


6. Hashing


hashing a string

In [1]:
ord('c')

99

In [2]:
ord('a')

97

In [3]:
ord('t')

116

In [17]:
def hash(a_string, table_size):
  sum = 0
  for pos in range(len(a_string)):
    sum = sum + pos*ord(a_string[pos])

  return sum % table_size